# Map

![domain decomposition](https://computing.llnl.gov/tutorials/parallel_comp/images/domain_decomp.gif)


## `map` function example

The `map(func, seq)` Python function applies the function func to all the elements of the sequence seq. It returns a new list with the elements changed by func

In [117]:
def f(x):
    return x * x

rdd = [2, 6, -3, 7]
res = map(f, rdd )
res  # Res is an iterator

In [118]:
print(*res)

4 36 9 49


In [119]:
def f(x, y):
    return x + y
rdd1, rdd2 = [2, 6, -3, 7], [1, -4, 5, 3]
res = map(f, rdd1, rdd2 ) # element wise sum of rdd1 and rdd2 

In [120]:
print(*res)

3 2 2 10


# Reduce

![MapReduce](http://mm-tom.s3.amazonaws.com/blog/MapReduce.png)

## `functools.reduce` example

The function `reduce(func, seq)` continually applies the function func() to the sequence seq and return a single value. For example, reduce(f, [1, 2, 3, 4, 5]) calculates f(f(f(f(1,2),3),4),5).

In [121]:
from functools import reduce
from operator import add

rdd = [1, 2, 3, 4, 5]
reduce(add, rdd) # computes ((((1+2)+3)+4)+5)

15

In [122]:
sum(rdd)

15

In [123]:
def g(x,y):
    return x * y

rdd = [1, 2, 3, 4, 5]
reduce(g, rdd) 

120

In [124]:
p = 1
for x in rdd:
    p *= x
p

120

## Weighted mean and Variance

If the generator of random variable $X$ is discrete with probability mass function $x_1 \mapsto p_1, x_2 \mapsto p_2, \ldots, x_n \mapsto p_n$ then

$$\operatorname{Var}(X) = \left(\sum_{i=1}^n p_i x_i ^2\right) - \mu^2,$$

where $\mu$ is the average value, i.e.

$$\mu = \sum_{i=1}^n p_i x_i. $$

In [125]:
X = [5, 1, 2, 3, 1, 2, 5, 4]
P = [0.05, 0.05, 0.15, 0.05, 0.15, 0.2, 0.1, 0.25]

### Exercise 5.1

- Write functions to compute the average value and variance using for loops

In [126]:
def average(X, P):
    """Compute the average value of X with weight P"""
    avg = 0
    for x, p in zip(X, P):
        avg += x*p
    return avg

average(X, P)

def average(X, P):
    """Compute the weighted average value of X with weight P"""
    avg = sum([x*p for x,p in zip(X, P)])
    return avg

average(X, P)

2.8

In [127]:
help(average)

Help on function average in module __main__:

average(X, P)
    Compute the weighted average value of X with weight P



In [128]:
def variance(X, P):
    """Compute the variance of X with weight P"""
    var = 0
    for x, p in zip(X, P):
        var += p*x*x
    var -= average(X, P)**2
    return var

variance(X, P)

def variance(X, P):
    """Compute the variance of X with weight P"""
    mu = average(X, P)
    return(sum([p*x*x for x, p in zip(X,P)]) - mu**2)

variance(X, P)

1.9600000000000017

### Exercise 5.2

- Write functions to compute the average value and variance using `map` and `reduce`

In [5]:
from functools import reduce
from operator import add, mul

def average_mr(X, P):
    """Compute the weighted average value of X with weight P using map and reduce method"""
    mapped = map(mul, X, P)
    avg = reduce(add, mapped)
    return avg

average_mr(X, P)
    

NameError: name 'X' is not defined

In [6]:
def variance_mr(X, P):
    """Compute the variance of X with weight P using map and reduce method"""
    mu = average_mr(X, P)
    mapped = map(lambda x,p:p*x*x, X, P)
    return(reduce(add, mapped) - mu**2)
    
variance_mr(X, P)

NameError: name 'X' is not defined

*NB: Exercises above are just made to help to understand map-reduce process.
This is a bad way to code a variance in Python. You should use [Numpy](http://www.numpy.org) instead.*

## Wordcount 

We will modify the `wordcount` application into a map-reduce process.

The `map` process takes text files as input and breaks it into words. The `reduce`  process sums the counts for each word and emits a single key/value with the word and sum.

We need to split the wordcount function we wrote in notebook 04 in order to use map and reduce. 

In the following exercices we will implement in Python the Java example described in [Hadoop documentation](https://hadoop.apache.org/docs/current/hadoop-mapreduce-client/hadoop-mapreduce-client-core/MapReduceTutorial.html#Example:_WordCount_v1.0).

# Map 

## Read file and return a key/value pairs

### Exercise 5.3

Write a function `mapper` with a single file name as input that returns a sorted sequence of tuples (word, 1) values.

```pybt
mapper('sample.txt')
[('adipisci', 1), ('adipisci', 1), ('adipisci', 1), ('adipisci', 1), ('adipisci', 1), ('adipisci', 1), ('adipisci', 1), ('aliquam', 1), ('aliquam', 1), ('aliquam', 1), ('aliquam', 1), ('aliquam', 1), ('aliquam', 1), ('aliquam', 1), ('amet', 1), ('amet', 1), ('amet', 1)...
```

In [14]:
def mapper(filename):
    """Function with a single file name as input that returns a sorted sequence of tuples (word, 1) values."""
    with open(filename, 'r') as f:
        text = f.read()
    word_list = sorted(text.lower().replace('.', '').split())
    return [(word, 1) for word in word_list]

mapper("sample1.txt")    

FileNotFoundError: [Errno 2] No such file or directory: 'sample1.txt'

# Partition

### Exercise 5.4

Create a function named `partitioner` that stores the key/value pairs from `mapper`  that group (word, 1) pairs into a list as:
```python
partitioner(mapper('sample.txt'))
[('adipisci', [1, 1, 1, 1, 1, 1, 1]), ('aliquam', [1, 1, 1, 1, 1, 1, 1]), ('amet', [1, 1, 1, 1],...]
```

In [8]:
from collections import defaultdict

def partitioner(mapper_res):
    """Function that stores the key/value pairs from mapper that group (word, 1) pairs into a list
    of format (word, [1, 1, 1, ...] with n times 1, n is the number of occurences of word)"""
    word_dict = defaultdict(list)
    for word, occ in mapper_res:
        word_dict[word].append(occ)
    return sorted(list(word_dict.items()))

partitioner(mapper('sample1.txt'))

[('adipisci', [1, 1]),
 ('aliquam', [1, 1, 1, 1, 1, 1]),
 ('amet', [1, 1, 1, 1]),
 ('consectetur', [1, 1, 1]),
 ('dolor', [1, 1, 1, 1]),
 ('dolore', [1, 1, 1, 1, 1, 1]),
 ('dolorem', [1, 1, 1, 1, 1, 1]),
 ('eius', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 ('est', [1, 1, 1, 1, 1]),
 ('etincidunt', [1, 1, 1]),
 ('ipsum', [1, 1, 1, 1]),
 ('labore', [1, 1]),
 ('magnam', [1, 1, 1, 1, 1, 1]),
 ('modi', [1, 1, 1, 1, 1, 1]),
 ('neque', [1, 1, 1, 1, 1]),
 ('non', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 ('numquam', [1, 1, 1]),
 ('porro', [1, 1, 1, 1, 1, 1, 1]),
 ('quaerat', [1, 1, 1, 1]),
 ('quiquia', [1, 1, 1, 1, 1]),
 ('quisquam', [1]),
 ('sed', [1, 1]),
 ('sit', [1, 1, 1, 1]),
 ('tempora', [1, 1, 1, 1, 1, 1]),
 ('ut', [1, 1, 1, 1]),
 ('velit', [1, 1, 1, 1]),
 ('voluptatem', [1, 1, 1, 1, 1, 1])]

# Reduce 

## Sums the counts and returns a single key/value (word, sum).

### Exercice 5.5

Write the function `reducer` that read a tuple (word,[1,1,1,..,1]) and sum the occurrences of word to a final count, and then output the tuple (word,occurences).

```python
reducer(('hello',[1,1,1,1,1])
('hello',5)
```

In [9]:
def reducer(partition_elmt):
    """Function that read a tuple (word,[1,1,1,..,1]) and sum the occurrences of word to a final count,
    and then output the tuple (word,occurences)."""
    return (partition_elmt[0], len(partition_elmt[1]))

reducer(('hello', [1,1,1,1,1]))

('hello', 5)

# Process several files

Let's create 8 files `sample[0-7].txt`. Set most common words at the top of the output list.

In [10]:
from lorem import text
for i in range(8):
    with open("sample{0:02d}.txt".format(i), "w") as f:
        f.write(text())

In [11]:
import glob
files = glob.glob('sample0*.txt')
files

['sample00.txt',
 'sample01.txt',
 'sample02.txt',
 'sample03.txt',
 'sample04.txt',
 'sample05.txt',
 'sample06.txt',
 'sample07.txt']

### Exercise 5.6
- Use functions implemented above to count (word, occurences) by using a for loops over files and partitioned data.

In [13]:
def wordcount(file_list):
    """Function taking for argument a list of files and returns the count (word, occurences)
    of all files together sorted by the number of occurences"""
    mapped, res = [], []
    for file in file_list:
        mapped += mapper(file)
    
    for i in partitioner(mapped):
        res.append(reducer(i))
    return sorted(res, key=lambda t:t[1], reverse=True)

wordcount(files)

Wall time: 4.01 ms


### Exercise 5.7
- This time use `map` function to apply mapper and reducer.


In [137]:
from functools import reduce
from itertools import chain

def wordcount(file_list):
    """Function taking for argument a list of files and returns the count (word, occurences)
    of all files together sorted by the number of occurences"""
    mapped = map(mapper, file_list)
    res = map(reducer, partitioner(chain(*mapped)))
    return sorted(res, key=lambda t:t[1], reverse=True)

wordcount(files)

[('magnam', 85),
 ('tempora', 85),
 ('ipsum', 75),
 ('modi', 75),
 ('porro', 74),
 ('aliquam', 73),
 ('quisquam', 72),
 ('ut', 71),
 ('est', 70),
 ('labore', 70),
 ('quiquia', 70),
 ('dolor', 67),
 ('eius', 67),
 ('sit', 67),
 ('etincidunt', 66),
 ('voluptatem', 65),
 ('numquam', 64),
 ('quaerat', 64),
 ('dolorem', 63),
 ('non', 63),
 ('sed', 63),
 ('amet', 62),
 ('velit', 61),
 ('adipisci', 60),
 ('consectetur', 59),
 ('neque', 59),
 ('dolore', 48)]